In [1]:
import numpy as np
import tensornetwork as tn
import tensorflow as tf
import numba
import json

# Layers

In [2]:
class QuantumDMRGLayer(tf.keras.layers.Layer):
    def __init__(self, dimvec, pos_label, nblabels, bond_len, nearzero_std=1e-9, isolated_labelnode=True):
        super(QuantumDMRGLayer, self).__init__()
        self.dimvec = dimvec
        self.pos_label = pos_label
        self.nblabels = nblabels
        self.m = bond_len
        self.isolated_label = isolated_labelnode

        assert self.pos_label >= 0 and self.pos_label < self.dimvec

        self.mps_tensors = [tf.Variable(self.mps_tensor_initial_values(i, nearzero_std=nearzero_std),
                                        trainable=True,
                                        name='mps_tensors_{}'.format(i))
                            for i in range(self.dimvec)]
        if self.isolated_label:
            self.output_tensor = tf.Variable(tf.random.normal((self.m, self.m, self.nblabels),
                                                              mean=0.0,
                                                              stddev=nearzero_std),
                                             trainable=True,
                                             name='mps_output_node')

    def mps_tensor_initial_values(self, idx, nearzero_std=1e-9):
        if idx == 0 or idx == self.dimvec - 1:
            tempmat = tf.eye(max(2, self.m))
            mat = tempmat[0:2, :] if 2 < self.m else tempmat[:, 0:self.m]
            return mat + tf.random.normal(mat.shape, mean=0.0, stddev=nearzero_std)
        elif not self.isolated_label and idx == self.pos_label:
            return tf.random.normal((2, self.m, self.m, self.nblabels),
                                    mean=0.0,
                                    stddev=nearzero_std)
        else:
            return tf.random.normal((2, self.m, self.m),
                                    mean=0.0,
                                    stddev=nearzero_std)

    def infer_single(self, input):
        assert input.shape[0] == self.dimvec
        assert input.shape[1] == 2

        nodes = [
            tn.Node(self.mps_tensors[i], backend='tensorflow')
            for i in range(self.dimvec)
        ]
        if self.isolated_label:
            output_node = tn.Node(self.output_tensor, backend='tensorflow')
        input_nodes = [
            tn.Node(input[i, :], backend='tensorflow')
            for i in range(self.dimvec)
        ]

        for i in range(self.dimvec):
            nodes[i][0] ^ input_nodes[i][0]
        if self.isolated_label:
            nodes[0][1] ^ nodes[1][1]
            for i in range(1, self.pos_label):
                nodes[i][2] ^ nodes[i + 1][1]
            nodes[self.pos_label][2] ^ output_node[0]
            output_node[1] ^ nodes[self.pos_label + 1][1]
            for i in range(self.pos_label + 1, self.dimvec - 1):
                nodes[i][2] ^ nodes[i + 1][1]
        else:
            nodes[0][1] ^ nodes[1][1]
            for i in range(1, self.dimvec-1):
                nodes[i][2] ^ nodes[i + 1][1]

        if self.isolated_label:
            final_node = tn.contractors.auto(nodes + input_nodes + [output_node],
                                             output_edge_order=[output_node[2]])
        else:
            final_node = tn.contractors.auto(nodes + input_nodes,
                                             output_edge_order=[nodes[self.pos_label][3]])
        return final_node.tensor

    def call(self, inputs):
        return tf.vectorized_map(self.infer_single, inputs)

# Hyperparameters

In [3]:
 # model parameters
dimvec = 4
pos_label = 2
nblabels = 4
nbdata = 1000

# training and CV parameters
nb_epochs = 1000
cv_fold = 5
batch_size = 10
std = 1e-9
learning_rate = 1e-2
bond_len = 10

# Generating Data

In [4]:
def generate_random_data(size, std=0.2):
    centroids = {0: np.array([0.5, 0.5, 0, 1]), 
                 1: np.array([-0.5, 0.5, 0, -1]),
                 2: np.array([-0.5, -0.5, 1, 0]),
                 3: np.array([0.5, -0.5, -1, 0])}
    for _ in range(size):
        quadrant = np.random.choice(range(4))
        yield np.random.normal(loc=centroids[quadrant], scale=std), quadrant
        

@numba.njit(numba.float64[:, :](numba.float64[:]))
def convert_pixels_to_tnvector(vector):
    tnvector = np.concatenate((
        np.array([[vector[0], np.sign(vector[0])*(1-np.abs(vector[0]))]]),
        np.array([[vector[1], np.sign(vector[1])*(1-np.abs(vector[1]))]]),
        np.array([[vector[2], np.sign(vector[2])*(1-np.abs(vector[0]))]]),
        np.array([[vector[3], np.sign(vector[1])*(1-np.abs(vector[1]))]])

    ), axis=0)
    return tnvector


In [5]:
size = 1000

X = np.zeros((size, 4, 2))
Y = np.zeros((size, 4))

for i, (thisX, thisY) in enumerate(generate_random_data(size)):
    X[i, :, :] = convert_pixels_to_tnvector(thisX)
    Y[i, thisY] = 1.

In [6]:
# Prepare for cross-validation
cv_labels = np.random.choice(range(cv_fold), size=nbdata)

# Test

In [7]:
simple_nn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(dimvec, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])
simple_nn.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), 
                  loss=tf.keras.losses.CategoricalCrossentropy())

In [8]:
simple_nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 64)                576       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 836
Trainable params: 836
Non-trainable params: 0
_________________________________________________________________


In [9]:
simple_nn.fit(X[cv_labels==0, :, :], Y[cv_labels==0, :], epochs=100, verbose=0)

In [10]:
output = simple_nn.predict(X[cv_labels==0, :, :])

In [11]:
simple_nn.evaluate(X[cv_labels==0, :, :], Y[cv_labels==0, :])

212/212 [==============================] - 0s 394us/sample - loss: 0.5024


0.5024276645678394

In [12]:
np.sum(np.argmax(output, axis=1)==np.argmax(Y[cv_labels==0], axis=1))

211

In [13]:
np.sum(cv_labels==0)

212

# Model

In [14]:
def QuantumKerasModel(dimvec, pos_label, nblabels, bond_len, nearzero_std=1e-9, optimizer='adam'):
    quantum_dmrg_model = tf.keras.Sequential([
        tf.keras.Input(shape=(dimvec, 2)),
        QuantumDMRGLayer(dimvec=dimvec,
                         pos_label=pos_label,
                         nblabels=nblabels,
                         bond_len=bond_len,
                         nearzero_std=nearzero_std),
#         tf.keras.layers.LayerNormalization(beta_initializer='RandomUniform', gamma_initializer='RandomUniform', beta_constraint='non_neg')
        tf.keras.layers.LayerNormalization(beta_initializer='RandomUniform', gamma_initializer='RandomUniform'),
        tf.keras.layers.Softmax()
    ])
    quantum_dmrg_model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
    return quantum_dmrg_model

In [15]:
cv_idx = 0

trainX = X[cv_labels==cv_idx, :, :]
trainY = Y[cv_labels==cv_idx, :]

In [16]:
# training and CV parameters
nb_epochs = 10000
batch_size = 10
std = 1e-2
learning_rate = 1e-3
bond_len = 10

In [17]:
quantum_dmrg_model = QuantumKerasModel(dimvec=dimvec, pos_label=pos_label, nblabels=nblabels, bond_len=bond_len,
                                       nearzero_std=std, optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

In [18]:
quantum_dmrg_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantum_dmrg_layer (QuantumD (None, 4)                 840       
_________________________________________________________________
layer_normalization (LayerNo (None, 4)                 8         
_________________________________________________________________
softmax (Softmax)            (None, 4)                 0         
Total params: 848
Trainable params: 848
Non-trainable params: 0
_________________________________________________________________


In [19]:
# tn_layer = quantum_dmrg_model.layers[0]
# tn_layer.trainable_variables[0:3]

In [20]:
quantum_dmrg_model.fit(trainX, trainY, epochs=nb_epochs, batch_size=batch_size, verbose=0)

In [21]:
output = quantum_dmrg_model.predict(trainX)

In [22]:
quantum_dmrg_model.evaluate(trainX, trainY)

212/212 [==============================] - 0s 1ms/sample - loss: 0.6156


0.6155790281745622

In [23]:
# tn_layer.trainable_variables[0:3]

In [24]:
np.sum(np.argmax(output, axis=1)==np.argmax(Y[cv_labels==0, :], axis=1))

141

In [25]:
np.sum(cv_labels==0)

212